# Explore here

In [31]:
# Your code here
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import *
from imblearn.metrics import specificity_score
from xgboost import XGBClassifier



In [32]:
df = pd.read_csv('/workspaces/pauline-BOOSTING-ALGORITHMS/data/raw/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [33]:
#EDA : https://github.com/4GeeksAcademy/pauline-DECISION-TREE-ALGO.
#Pero la completamos borrando valores ceros que no son compatibles : 

#Descripción básica :
df.describe().T.round(1)


,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,3.8,3.4,0.0,1.0,3.0,6.0,17.0
Glucose,768.0,120.9,32.0,0.0,99.0,117.0,140.2,199.0
BloodPressure,768.0,69.1,19.4,0.0,62.0,72.0,80.0,122.0
SkinThickness,768.0,20.5,16.0,0.0,0.0,23.0,32.0,99.0
Insulin,768.0,79.8,115.2,0.0,0.0,30.5,127.2,846.0
BMI,768.0,32.0,7.9,0.0,27.3,32.0,36.6,67.1
DiabetesPedigreeFunction,768.0,0.5,0.3,0.1,0.2,0.4,0.6,2.4
Age,768.0,33.2,11.8,21.0,24.0,29.0,41.0,81.0
Outcome,768.0,0.3,0.5,0.0,0.0,0.0,1.0,1.0


In [34]:
#Glucoses, bloodpressure, skinthinckness, insulin y BMI no pueden ser exactamente 0, remplazamos los 0 x mean de la columna :

df['Glucose'] = df['Glucose'].replace(0, df['Glucose'].mean())
df['BloodPressure'] = df['BloodPressure'].replace(0, df['BloodPressure'].mean())
df['SkinThickness'] = df['SkinThickness'].replace(0, df['SkinThickness'].mean())
df['Insulin'] = df['Insulin'].replace(0, df['Insulin'].mean())
df['BMI'] = df['BMI'].replace(0, df['BMI'].mean())


In [35]:
#Verificamos los min:
df.describe().T.round(1)

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,3.8,3.4,0.0,1.0,3.0,6.0,17.0
Glucose,768.0,121.7,30.4,44.0,99.8,117.0,140.2,199.0
BloodPressure,768.0,72.3,12.1,24.0,64.0,72.0,80.0,122.0
SkinThickness,768.0,26.6,9.6,7.0,20.5,23.0,32.0,99.0
Insulin,768.0,118.7,93.1,14.0,79.8,79.8,127.2,846.0
BMI,768.0,32.5,6.9,18.2,27.5,32.0,36.6,67.1
DiabetesPedigreeFunction,768.0,0.5,0.3,0.1,0.2,0.4,0.6,2.4
Age,768.0,33.2,11.8,21.0,24.0,29.0,41.0,81.0
Outcome,768.0,0.3,0.5,0.0,0.0,0.0,1.0,1.0


In [36]:
#Separamos train y test
X = df.drop(['Outcome'], axis=1)
y = df['Outcome']

#Lo convertimos a numpy array sino me daba error despues:
X = X.values  
y = y.values 


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [37]:
#Misma función que el proyecto anterior para ver resultados :
 
def get_metrics(y_train, y_test, y_pred_train, y_pred_test):
    # Calcular métricas para el conjunto de entrenamiento
    train_accuracy = accuracy_score(y_train, y_pred_train)
    train_f1 = f1_score(y_train, y_pred_train)
    train_auc = roc_auc_score(y_train, y_pred_train)
    train_precision = precision_score(y_train, y_pred_train)
    train_recall = recall_score(y_train, y_pred_train)
    train_specificity = specificity_score(y_train, y_pred_train)

    # Calcular métricas para el conjunto de prueba
    test_accuracy = accuracy_score(y_test, y_pred_test)
    test_f1 = f1_score(y_test, y_pred_test)
    test_auc = roc_auc_score(y_test, y_pred_test)
    test_precision = precision_score(y_test, y_pred_test)
    test_recall = recall_score(y_test, y_pred_test)
    test_specificity = specificity_score(y_test, y_pred_test)

    # Calcular la diferencia entre métricas de entrenamiento y prueba
    diff_accuracy = train_accuracy - test_accuracy
    diff_f1 = train_f1 - test_f1
    diff_auc = train_auc - test_auc
    diff_precision = train_precision - test_precision
    diff_recall = train_recall - test_recall
    diff_specificity = train_specificity - test_specificity

    # Crear un DataFrame con los resultados
    metrics_df = pd.DataFrame([[train_accuracy, train_f1, train_auc, train_precision, train_recall, train_specificity],[test_accuracy, test_f1, test_auc, test_precision, test_recall, test_specificity],[diff_accuracy, diff_f1, diff_auc, diff_precision, diff_recall, diff_specificity]],
                              columns = ['Accuracy', 'F1', 'AUC', 'Precision', 'Recall', 'Specificity'],
                              index = ['Train','Test', 'Diferencia'])

    return metrics_df

In [38]:
#Algoritmo de boosting, primero vamos a ver con XGBClassifier

#Entrenamos el modelo :
model = XGBClassifier(n_estimators = 200, learning_rate = 0.001, random_state = 42)
model.fit(X_train, y_train)

# Realizamos predicciones en el conjunto de entrenamiento y prueba
test_pred = model.predict(X_test)
train_pred = model.predict(X_train)

# Evaluamos el rendimiento del modelo
get_metrics(y_train, y_test, train_pred, test_pred)

,Accuracy,F1,AUC,Precision,Recall,Specificity
Train,0.864821,0.811791,0.859091,0.785088,0.840376,0.877805
Test,0.740260,0.672131,0.741414,0.611940,0.745455,0.737374
Diferencia,0.124561,0.139660,0.117676,0.173147,0.094921,0.140432


In [42]:
#Algoritmo de boosting; ahora usamos el CustomBoosting visto en clase:
 
class CustomBoosting:

#El constructor : 
  def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=10, min_samples_leaf=10, max_features=1.0, random_state=42):
        self.n_estimators = n_estimators  # Número de arboles en el ensamble
        self.learning_rate = learning_rate  # Tasa de aprendizaje
        self.max_depth = max_depth  # Profundidad máxima de los árboles de decisión
        self.min_samples_leaf = min_samples_leaf  # Número mínimo de muestras por hoja
        self.max_features = max_features  # Proporción máxima de características a considerar en cada división
        self.random_state = random_state  # Semilla aleatoria para reproducibilidad
        self.estimadores = [] # Almacena los estimadores del ensamble


#Los 4 metodos que necesitamos para el fit :

#muestreo ponderado de los datos de entrada
  def muestreo(self, X, y, pesos):
    # Semilla para replicar el experimento
    np.random.seed(self.random_state)
    # Selecciono lo indices que serán usados para generar la muestra
    indices = np.random.choice(a=len(X), size=len(X), p=pesos)
    return X[indices], y[indices]


#error ponderado del modelo en los datos:
  def get_error(self, y, y_pred, pesos):
    # Obtener las instancias mal clasificadas
    instancias_erroneas = (y!=y_pred)
    # Obtenemos los pesos de las instancias erróneamente clasificadas
    pesos_instancias_erroneas = pesos[instancias_erroneas]
    # Suma total del peso de las instancias erróneas
    suma_pesos_error = np.sum(pesos_instancias_erroneas)
    # Normalizamos el error para que tome un valor entre 0 y 1
    error_ponderado = suma_pesos_error / np.sum(pesos)
    return instancias_erroneas, error_ponderado
  

#importancia del estimador en función de su error:

  def get_estimator_importance(self, error_ponderado):
    # Proporción de acierto frente al error
    return np.log((1-error_ponderado)/error_ponderado)*self.learning_rate
  

#actualiza los pesos de las instancias después de cada iteración:

  def update_pesos(self, estimator_importance, instancias_erroneas, pesos):
    # Actualiza los pesos de las instancias para dar más importancia a las instancias mal clasificadas
    # incrementar de forma más específica los pesos de las instancias que fueron mal clasificadas,
    # mientras que disminuye los de las que fueron correctamente clasificadas, ajustando así de manera más efectiva la atención del modelo hacia las instancias que más necesita aprender.
    actualizacion_pesos = np.where(instancias_erroneas, np.exp(estimator_importance), np.exp(-estimator_importance))
    pesos = pesos * actualizacion_pesos  # Aplica la actualización a los pesos
    pesos = pesos / np.sum(pesos)  # Normaliza los pesos para que sumen 1, creando una distribución de probabilidad válida
    return pesos




  def fit(self, X, y):
    # Inicializar los pesos de las instancias
    pesos = np.ones(len(X))/len(X) #para normalizar, al inicio el modelo no tiene ninguna preferencia
  
    # Realizamos el entrenamiento de los estimadores
    for _ in range(self.n_estimators):
      # Realizamos un muestreo del dataset teniendo en cuenta el peso asociado a cada instancia
      X_sampled, y_sampled = self.muestreo(X, y, pesos)
      # Entrenamos el estimador con la versión muestreada
      estimador = DecisionTreeClassifier(max_depth=self.max_depth, min_samples_leaf=self.min_samples_leaf, max_features=self.max_features).fit(X_sampled, y_sampled)
      # Realizamos predicciones con el estimador entrenado
      y_pred = estimador.predict(X)
      # Obtener el error cometido por el estimador
      instancias_erroneas, error_ponderado = self.get_error(y, y_pred, pesos)
      # Calcular la importancia del estimador
      estimator_importance = self.get_estimator_importance(error_ponderado)
      # Almacenar el estimador junto a su importancia
      self.estimadores.append((estimador, estimator_importance))
      # Actualizar los pesos de las instancias
      pesos = self.update_pesos(estimator_importance, instancias_erroneas, pesos)

  def predict(self, X):
    # Inicializamos una variable donde almacenar las predicciones finales
    final_predictions = np.zeros(len(X)) 
    # Para cada estimador del ensamblado
    for estimador, estimador_imp in self.estimadores:
      # Aplicar el estimador sobre los datos a predecir
      predicciones = estimador.predict(X)
      # Pondero las precciones
      preds_ponderada = predicciones*estimador_imp
      # Acumula las predicciones ponderadas
      final_predictions = final_predictions+preds_ponderada
    mean_val = np.mean(final_predictions)
    yhats = np.where(final_predictions<mean_val,0,1)
    return yhats


In [40]:
# Entrenamos el modelo de Boosting
clf = CustomBoosting(n_estimators=10, learning_rate=0.01, max_depth=7, min_samples_leaf=7, max_features=X_train.shape[1]//2)
clf.fit(X_train, y_train)

# Realizamos predicciones en el conjunto de entrenamiento y prueba
test_pred = clf.predict(X_test)
train_pred = clf.predict(X_train)

# Evaluamos el rendimiento del modelo
get_metrics(y_train, y_test, train_pred, test_pred)

,Accuracy,F1,AUC,Precision,Recall,Specificity
Train,0.850163,0.797357,0.850070,0.751037,0.849765,0.850374
Test,0.766234,0.714286,0.777778,0.633803,0.818182,0.737374
Diferencia,0.083929,0.083071,0.072292,0.117235,0.031583,0.113000


**Analisis de comparación de los 2 algoritmos:**

- **Accuracy**: XGBClassifier tiene mejor accuracy en el conjunto de train (86%) que el Custom Boosting (85).
Pero para test, el Custom Boosting está mejor (77% frente a 74%).

- **F1 Score, precision, recall, AUC:**: igual, para el conjunto de train, XGB tiene un rendimiento superior. Pero para el conjunto test, es CustomBoosting que está mejor.

- **Specificity**: XGB es superior para el conjunto train. Para test tienen el mismo resultado


**Conclusión**: El Custom Boosting parece generalizar mejor en los datos de prueba, entonces es mejor quedarnos con eso.